<a href="https://colab.research.google.com/github/darknight11345/turtlebot4/blob/main/Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
pwd

'/content'

In [ ]:
#generate dataset
import random
import pandas as pd
import re

# Define a list of possible words for the sentence generation
actions = ["Deliver", "Go", "Take", "Bring", "Move", "Send", "Head towards", "Proceed", "Navigate", "Transport"]
locations = ["ICU", "reception", "emergency room", "doctors room", "nurse station"]  # Include 'nurse station' as a location
rooms = [str(i) for i in range(101, 601)]  # Random room numbers between 101 and 600
medicines = ["Paracetamol", "Aspirin", "Ibuprofen", "Tylenol", "Penicillin", "Antibiotics", "Cough Syrup"]

# List of multi-word locations to handle separately
multi_word_locations = ["nurse station", "doctors room", "emergency room"]

# Function to generate random sentences with action and location
def generate_sentence_with_action():
    action = random.choice(actions)
    medicine = random.choice(medicines)



    # Special logic for actions like 'Navigate', 'Proceed', 'Head towards'
    if action in ["Navigate", "Proceed", "Head towards", "Move", "Go"]:
        if random.choice([True, False]):
            # Location-based sentence
            location = random.choice(locations)
            sentence = f"{action} to {location} with {medicine}."
        else:
            # Room number-based sentence
            room = random.choice(rooms)
            sentence = f"{action} to Room {room} with {medicine}."

    else:
           # 50% chance for either a location or a room number
        if random.choice([True, False]):
            location = random.choice(locations)
            sentence = f"{action} {medicine} to {location}."
        else:
            room = random.choice(rooms)  # Random room number between 101 and 600
            sentence = f"{action} {medicine} to Room {room}."

    # Clean the sentence by removing unwanted special characters (e.g., periods, commas, etc.)
    sentence = re.sub(r'[^\w\s]', '', sentence)

    return sentence

# Generate 1000 sentences with actions, without sentence breaks
data = []
sentence_id = 1
for _ in range(1000):
    sentence = generate_sentence_with_action()

    # Handle multi-word locations before splitting
    for multi_word_loc in multi_word_locations:
        if multi_word_loc in sentence:
            sentence = sentence.replace(multi_word_loc, f"{multi_word_loc.replace(' ', '_')}")

    tokens = sentence.split()

    # Process each token in the sentence
    i = 0
    while i < len(tokens):
        token = tokens[i]

        # Special handling for "Head towards" as a single ACTION entity
        if token == "Head" and i + 1 < len(tokens) and tokens[i + 1] == "towards":
            data.append((sentence_id, "Head towards", "ACTION"))
            i += 1  # Skip the next token ("towards")
        elif token in actions and token != "Head":
            data.append((sentence_id, token, "ACTION"))
        elif token in medicines:
            data.append((sentence_id, token, "MEDICINE"))
        elif token == "Room":
            # Combine "Room" and the next token (the number) as a LOCATION entity
            if i + 1 < len(tokens):
                room_number = f"Room {tokens[i+1]}"
                data.append((sentence_id, room_number, "LOCATION"))
                i += 1  # Skip the next token (room number)
        elif token.replace("_", " ") in multi_word_locations:
            # Replace underscores back to spaces for multi-word locations
            location = token.replace("_", " ")
            data.append((sentence_id, location, "LOCATION"))
        elif token in locations:
            data.append((sentence_id, token, "LOCATION"))  # Locations like ICU, reception, etc.
        else:
            data.append((sentence_id, token, "O"))  # Words that are not part of any entity

        i += 1

    sentence_id += 1

# Create DataFrame and save the dataset to CSV
df = pd.DataFrame(data, columns=["Sentence_ID", "Word", "Label"])
df.to_csv("ner_dataset_with_head_towards_handling.csv", index=False)

print("CSV file 'ner_dataset_with_head_towards_handling.csv' created successfully.")


In [ ]:
#setup of dataset

In [4]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00
   

In [5]:
!pip install fsspec==2024.10.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.


In [6]:
import pandas as pd
data = pd.read_csv("ner_dataset_with_head_towards_handling.csv",encoding="latin1" )

In [7]:
data.head(30)

,Sentence_ID,Word,Label
0,1,Deliver,ACTION
1,1,Tylenol,MEDICINE
2,1,to,O
3,1,Room 493,LOCATION
4,2,Go,ACTION
5,2,to,O
6,2,ICU,LOCATION
7,2,with,O
8,2,Aspirin,MEDICINE
9,3,Go,ACTION


In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
data.rename(columns={"Sentence_ID":"sentence_id","Word":"words","Label":"labels"}, inplace =True)

In [10]:
data["labels"] = data["labels"].str.upper()

In [11]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [13]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [14]:
test_data

,sentence_id,words,labels
3678,787,with,O
1348,290,Send,ACTION
4163,891,with,O
2772,592,Move,ACTION
3643,779,with,O
...,...,...,...
2631,561,to,O
2676,571,Room 220,LOCATION
1830,391,Take,ACTION
2636,562,to,O


In [15]:
train_data

,sentence_id,words,labels
289,62,with,O
3020,646,Head towards,ACTION
1436,308,Aspirin,MEDICINE
3038,649,Room 346,LOCATION
4457,954,Syrup,O
...,...,...,...
170,38,Head towards,ACTION
2264,483,to,O
2977,636,Room 102,LOCATION
878,188,to,O


In [16]:
#Model Training

In [17]:

!pip install torch  --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124


In [18]:


from simpletransformers.ner import NERModel,NERArgs

In [19]:
label = data["labels"].unique().tolist()
label

['ACTION', 'MEDICINE', 'O', 'LOCATION']

In [20]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32
args.gradient_accumulation_steps = 4  # Adjust this to your needs
args.max_seq_length = 128  # Adjust this according to your data



In [21]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [22]:
train_data

,sentence_id,words,labels
289,62,with,O
3020,646,Head towards,ACTION
1436,308,Aspirin,MEDICINE
3038,649,Room 346,LOCATION
4457,954,Syrup,O
...,...,...,...
170,38,Head towards,ACTION
2264,483,to,O
2977,636,Room 102,LOCATION
878,188,to,O


In [23]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)
#torch.cuda.empty_cache()

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

...

Running Epoch 7 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

(80, 0.09489480655370244)

In [35]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOCATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MEDICINE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ACTION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [39]:
#model.save_model("./text_trained_model_bert")
model.save_model()

In [25]:
result

{'eval_loss': 9.893858770651387e-05,
 'precision': 1.0,
 'recall': 1.0,
 'f1_score': 1.0}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [26]:
prediction, model_output = model.predict(["deploy tablets to 102 room"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
prediction

[[{'deploy': 'ACTION'},
  {'tablets': 'LOCATION'},
  {'to': 'O'},
  {'102': 'LOCATION'},
  {'room': 'LOCATION'}]]

In [28]:
model_output

[[{'deploy': [[7.951763, -1.932146, -2.4355783, -2.2769465]]},
  {'tablets': [[-1.3619403, 0.21120068, -1.9551146, 5.230475]]},
  {'to': [[-2.8606539, -2.7737563, 8.106287, -2.8383775]]},
  {'102': [[-1.1109443, -0.91380775, 0.84729266, 2.9554353]]},
  {'room': [[-2.5484447, -1.2569171, -2.4261367, 7.9173675]]}]]

In [32]:
#testing

# Load the trained model for inference
model1 = NERModel("bert", "./text_trained_model_bert",use_cuda=False)


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory ./text_trained_model_bert.

In [42]:
# Example text for prediction
text = ["bring tablet to ICU", "go to room 102"]

# Get predictions
predictions, raw_outputs = modeling_mobilenet_v1.predict(text)

# Print predictions
print(predictions)


NameError: name 'modeling_mobilenet_v1' is not defined